# Chapter 10: Introduction to Artificial Neural Networks with Keras

## 10.1 From Biological to Artificial Neurons

### 10.1.1 Biological Neurons

### 10.1.2 Logical Computations with Neurons

### 10.1.3 The Perceptron

### 10.1.4 The Multilayer Perceptron and Backpropagation

### 10.1.5 Regression MLPs

### 10.1.6 Classification MLPs

## 10.2 Implementing MLPs with Keras

### 10.2.1 Installing TensorFlow 2

### 10.2.2 Building an Image Classifier Using the Sequential API

### 10.2.3 Building a Regression MLP Using the Sequential API

### 10.2.4 Building Complex Models Using the Functional API

### 10.2.5 Using the Subclassing API to Build Dynamic Models

### 10.2.6 Saving and Restoring a Model

### 10.2.7 Using Callbacks

### 10.2.8 Using TensorBoard for Visualization

## 10.3 Fine-Tuning Neural Networks Hyperparameters

### 10.3.1 Number of Hidden Layers

### 10.3.2 Number of Neurons per Hidden Layer

### 10.3.3 Learning Rate, Batch Size, and Other Hyperparameters